In [2]:
import numpy as np
import math

In [3]:
#Area of triangle defined with parameter points
def area(x1, y1, x2, y2, x3, y3):
    return abs((x1 * (y2 - y3) + x2 * (y3 - y1)
                + x3 * (y1 - y2)) / 2.0)

#Function which returns True when point (x,y) is inside the triangle defines with points 1,2,3
def isInside(x1, y1, x2, y2, x3, y3, x, y):

    # Calculate area of triangle ABC
    A = area (x1, y1, x2, y2, x3, y3)
 
    # Calculate area of triangle PBC
    A1 = area (x, y, x2, y2, x3, y3)
     
    # Calculate area of triangle PAC
    A2 = area (x1, y1, x, y, x3, y3)
     
    # Calculate area of triangle PAB
    A3 = area (x1, y1, x2, y2, x, y)
     
    # Check if sum of A1, A2 and A3
    # is same as A
    if(A == A1 + A2 + A3):
        return True
    else:
        return False

#Vector abs
def absolute_vector(x):
    return math.sqrt(x[0]**2+x[1]**2)

In [4]:
#Next are all the projection functions
def pd1(x,y):
    if x**2+y**2<=1.5:
        return x,y
    else:
        return ( x*math.sqrt(1.5)/math.sqrt(x**2+y**2),y*math.sqrt(1.5)/math.sqrt(x**2+y**2) )
    
def pd2_map_single(x):
    if abs(x)<=1:
        return x
    elif x>1:
        return 1
    else:
        return -1
    
def pd2(x,y):
    if abs(x)<=1 and abs(y)<=1:
        return (x,y)
    else:
        return (pd2_map_single(x), pd2_map_single(y))


def pd3(x,y):
    if -1<=x and x<= 1.5 and -1<=y and y<= 1.5 and y<=0.5-x:
        return (x,y)
    elif -1<=y and y<=1.5 and x<-1:
        return (-1,y)
    elif -1<=x and x<=1.5 and y<-1:
        return (x,-1)
    elif x<-1 and y<-1:
        return (-1,-1)
    elif y>1.5 and y>x+2.5:
        return (-1,1.5)
    elif x>1.5 and y<x-2.5:
        return (1.5, -1)
    else:
        return (1/2*(x-y+0/5), 1/2*(1/2+y-x))

In [5]:
#Function which we are optimizing
def f(X):
    x=X[0]; y=X[1]
    return x**2+math.exp(x)+y**2-x*y

#The gradient of the function evaluated at x,y
def gradient_step(x,y):
    return np.array([2*x+math.exp(x)-y,2*y-x])

## Case 1_1:
This case uses the circle constrain domain and the fact that f is L-lipschitz.

$D: x^2+y^2<=1.5$

$\gamma=\frac{|x_{1}-x_{*}|}{L\sqrt{T}}$

In [6]:
x=np.array([-1,1])
x_star=np.array([-0.432563, -0.216281])

T=11
L=14.67
gamma=absolute_vector(x-x_star)/(L*math.sqrt(T))

all_points11=[x]
for i in range(1,11):
    x_new=x-gamma*np.array(gradient_step(x[0],x[1]))
    x=np.array(pd1( x_new[0], x_new[1] ))
    all_points11.append(x)
#all_points11

In [7]:
print(f'Actuall best is: {x_star}, PGD returned: {all_points11[10]}, difference in f: {f(x_star)-f(all_points11[10])}.')

Actuall best is: [-0.432563 -0.216281], PGD returned: [-0.50593927  0.3841299 ], difference in f: -0.41163867582559155.


## Case 1_2:
This case uses the circle constrain domain and the fact that f is $\beta$-smooth.

$D: x^2+y^2<=1.5$ 

$\gamma=\frac{1}{\beta}$

In [8]:
x=np.array([-1,1])

beta=9.522
gamma=1/beta

all_points12=[x]
for i in range(1,11):
    x_new=x-gamma*np.array(gradient_step(x[0],x[1]))
    x=np.array(pd1( x_new[0], x_new[1] ))
    all_points12.append(x)
#all_points12

In [9]:
print(f'Actuall best is: {x_star}, PGD returned: {all_points12[10]}, difference in f: {f(x_star)-f(all_points12[10])}')

Actuall best is: [-0.432563 -0.216281], PGD returned: [-0.3556857  -0.08413374], difference in f: -0.015181412782638182


## Case 1_3:
This case uses the circle constrain domain and the fact that f is $\alpha$-strongly convex and L-lipschitz.

$D: x^2+y^2<=1.5$ 

$\gamma_{k}=\frac{2}{\alpha(k+1)}$

In [10]:
x=np.array([-1,1])

beta=9.522
alpha=1.07

all_points13=[x]
for i in range(1,11):
    gamma=2/(alpha*(i+1))
    x_new=x-gamma*np.array(gradient_step(x[0],x[1]))
    x=np.array(pd1( x_new[0], x_new[1] ))
    all_points13.append(x)

In [11]:
print(f'Actuall best is: {x_star}, PGD returned: {all_points13[10]}, difference in f: {f(x_star)-f(all_points13[10])}')

Actuall best is: [-0.432563 -0.216281], PGD returned: [-0.43406103 -0.21835227], difference in f: -4.15856668500858e-06


For the circle domain the adaptive gamma returns the best results!

## Case 2_1:
This case uses the square constrain domain and the fact that f is L-lipschitz.

$D: [-1,1]x[-1,1]$

$\gamma=\frac{|x_{1}-x_{*}|}{L\sqrt{T}}$

In [12]:
x=np.array([-1,1])

T=11
L=14.67
gamma=absolute_vector(x-x_star)/(L*math.sqrt(T))

all_points21=[x]
for i in range(1,11):
    x_new=x-gamma*np.array(gradient_step(x[0],x[1]))
    x=np.array(pd2( x_new[0], x_new[1] ))
    all_points21.append(x)

In [13]:
print(f'Actuall best is: {x_star}, PGD returned: {all_points21[10]}, difference in f: {f(x_star)-f(all_points21[10])}')

Actuall best is: [-0.432563 -0.216281], PGD returned: [-0.52831125  0.41020396], difference in f: -0.46451820682723055


## Case 2_2:
This case uses the square constrain domain and the fact that f is $\beta$-smooth.

$D: [-1,1]x[-1,1]$

$\gamma=\frac{1}{\beta}$

In [14]:
x=np.array([-1,1])

beta=9.522
gamma=1/beta

all_points22=[x]
for i in range(1,11):
    x_new=x-gamma*np.array(gradient_step(x[0],x[1]))
    x=np.array(pd2( x_new[0], x_new[1] ))
    all_points22.append(x)

In [15]:
print(f'Actuall best is: {x_star}, PGD returned: {all_points22[10]}, difference in f: {f(x_star)-f(all_points22[10])}')

Actuall best is: [-0.432563 -0.216281], PGD returned: [-0.3556857  -0.08413374], difference in f: -0.015181412782638182


## Case 2_3:
This case uses the square constrain domain and the fact that f is $\alpha$-strongly convex and L-lipschitz.
$D: [-1,1]x[-1,1]$

$\gamma_{k}=\frac{2}{\alpha(k+1)}$

In [16]:
x=np.array([-1,1])

beta=9.522
alpha=1.07

all_points23=[x]
for i in range(1,11):
    gamma=2/(alpha*(i+1))
    x_new=x-gamma*np.array(gradient_step(x[0],x[1]))
    x=np.array(pd2( x_new[0], x_new[1] ))
    all_points23.append(x)

In [17]:
print(f'Actuall best is: {x_star}, PGD returned: {all_points23[10]}, difference in f: {f(x_star)-f(all_points23[10])}')

Actuall best is: [-0.432563 -0.216281], PGD returned: [-0.42753403 -0.20936829], difference in f: -4.6532596709281115e-05


For this domain , yet again the adaptive gamma is the best.

## Case 3_1:
This case uses the triangle constrain domain and the fact that f is L-lipschitz.

D: Triangle defined with (-1,1.5), (1.5,-1), (-1,-1)

$\gamma=\frac{|x_{1}-x_{*}|}{L\sqrt{T}}$

In [18]:
x=np.array([-1,1])

T=11
L=14.67
gamma=absolute_vector(x-x_star)/(L*math.sqrt(T))

all_points31=[x]
for i in range(1,11):
    x_new=x-gamma*np.array(gradient_step(x[0],x[1]))
    x=np.array(pd3( x_new[0], x_new[1] ))
    all_points31.append(x)

In [19]:
print(f'Actuall best is: {x_star}, PGD returned: {all_points31[10]}, difference in f: {f(x_star)-f(all_points31[10])}')

Actuall best is: [-0.432563 -0.216281], PGD returned: [-0.52831125  0.41020396], difference in f: -0.46451820682723055


## Case 3_2:
This case uses the triangle constrain domain and the fact that f is $\beta$-smooth.

D: Triangle defined with (-1,1.5), (1.5,-1), (-1,-1)

$\gamma=\frac{1}{\beta}$

In [20]:
x=np.array([-1,1])

beta=9.522
gamma=1/beta

all_points32=[x]
for i in range(1,11):
    x_new=x-gamma*np.array(gradient_step(x[0],x[1]))
    x=np.array(pd3( x_new[0], x_new[1] ))
    all_points32.append(x)

In [21]:
print(f'Actuall best is: {x_star}, PGD returned: {all_points32[10]}, difference in f: {f(x_star)-f(all_points32[10])}')

Actuall best is: [-0.432563 -0.216281], PGD returned: [-0.3556857  -0.08413374], difference in f: -0.015181412782638182


## Case 3_3:

This case uses the triangle constrain domain and the fact that f is $\alpha$-strongly convex and L-lipschitz.

D: Triangle defined with (-1,1.5), (1.5,-1), (-1,-1)

$\gamma_{k}=\frac{2}{\alpha(k+1)}$

In [22]:
x=np.array([-1,1])

beta=9.522
alpha=1.07

all_points33=[x]
for i in range(1,11):
    gamma=2/(alpha*(i+1))
    x_new=x-gamma*np.array(gradient_step(x[0],x[1]))
    x=np.array(pd3( x_new[0], x_new[1] ))
    all_points33.append(x)

In [23]:
print(f'Actuall best is: {x_star}, PGD returned: {all_points33[10]}, difference in f: {f(x_star)-f(all_points33[10])}')

Actuall best is: [-0.432563 -0.216281], PGD returned: [-0.4253522  -0.20635954], difference in f: -9.58010770396589e-05


One more time the fourth equation shows as the best.

# Theoretical guarantees vs actual obtained results

In the following part I calculated the theoretical guarantees from Theorem 3.3 from the lecture notes and calculated the left part of the inequations for all corresponding 9 combinations performed. Essentialy, the smaller the left part the smaller minimum we obtain.

## First inequality

In [25]:
ineq1=L*absolute_vector(all_points11[0] - x_star)/math.sqrt(11)
left_11=f(np.sum(all_points11, axis=0)/11)-f(x_star)
left_21=f(np.sum(all_points21, axis=0)/11)-f(x_star)
left_31=f(np.sum(all_points31, axis=0)/11)-f(x_star)
print(f'Equation guarantee is: {ineq1}, the left side of the equation for all three domains are:\n \
    -circle domain: {left_11},\n \
    -square domain: {left_21},\n \
    -triangular domain: {left_31}.')

Equation guarantee is: 5.9364896569245476, the left side of the equation for all three domains are:
     -circle domain: 1.0321992420877533,
     -square domain: 1.1577968902705496,
     -triangular domain: 1.1577968902705496.


We are smaller than the guarantee and the circle domain returnes the smallest margin!

## Second inequality


In [26]:
beta=9.522
ineq2=(3*beta*absolute_vector(all_points11[0] - x_star)**2+f(all_points11[0])-f(x_star))/11
left_12=f(all_points12[10])-f(x_star)
left_22=f(all_points22[10])-f(x_star)
left_32=f(all_points32[10])-f(x_star)
print(f'Equation guarantee is: {ineq2}, the left side of the equation for all three domains are:\n \
    -circle domain: {left_12},\n \
    -square domain: {left_22},\n \
    -triangular domain: {left_32}.')

Equation guarantee is: 4.912302733753495, the left side of the equation for all three domains are:
     -circle domain: 0.015181412782638182,
     -square domain: 0.015181412782638182,
     -triangular domain: 0.015181412782638182.


All of the domains return pretty much the same result.

## Fourth inequality

In [27]:
alpha=1.07
L=14.67
T=11
ineq3=(2*L**2)/(alpha*(12))
left_13=f( sum([2*(i+1)/(11*(11+1))*xi for i,xi in enumerate(all_points13)]) )-f(x_star)
left_23=f( sum([2*(i+1)/(11*(11+1))*xi for i,xi in enumerate(all_points23)]) )-f(x_star)
left_33=f( sum([2*(i+1)/(11*(11+1))*xi for i,xi in enumerate(all_points33)]) )-f(x_star)
print(f'Equation guarantee is: {ineq3}, the left side of the equation for all three domains are:\n \
    -circle domain: {left_13},\n \
    -square domain: {left_23},\n \
    -triangular domain: {left_33}.')

Equation guarantee is: 33.52163551401869, the left side of the equation for all three domains are:
     -circle domain: 6.177207223823089e-05,
     -square domain: 0.003365609811613046,
     -triangular domain: 0.007247069159252217.


Fourth equation underestimates by a lot, but square is the best!

## How close did we get to x*
In the following part, I calculated how close all 9 combinations came to x*. Essentialy, I calculated $|x_{11}-x_{*}|$ for all final points obtained from all PGDs.

In [28]:
print(f'How close did we get on the circular domain:\n \
    -using learning rate from the first equality:{absolute_vector(all_points11[10]-x_star)},\n \
    -using learning rate from the second equality:{absolute_vector(all_points12[10]-x_star)},\n \
    -using learning rate from the fourth equality:{absolute_vector(all_points13[10]-x_star)}.')

print(f'How close did we get on the squared domain:\n \
    -using learning rate from the first equality:{absolute_vector(all_points21[10]-x_star)},\n \
    -using learning rate from the second equality:{absolute_vector(all_points22[10]-x_star)},\n \
    -using learning rate from the fourth equality:{absolute_vector(all_points23[10]-x_star)}.')

print(f'How close did we get on the triangular domain:\n \
    -using learning rate from the first equality:{absolute_vector(all_points31[10]-x_star)},\n \
    -using learning rate from the second equality:{absolute_vector(all_points32[10]-x_star)},\n \
    -using learning rate from the fourth equality:{absolute_vector(all_points33[10]-x_star)}.')

How close did we get on the circular domain:
     -using learning rate from the first equality:0.6048779415773553,
     -using learning rate from the second equality:0.15288236795052068,
     -using learning rate from the fourth equality:0.0025562195781356854.
How close did we get on the squared domain:
     -using learning rate from the first equality:0.6337595179436561,
     -using learning rate from the second equality:0.15288236795052068,
     -using learning rate from the fourth equality:0.008548453592554486.
How close did we get on the triangular domain:
     -using learning rate from the first equality:0.6337595179436561,
     -using learning rate from the second equality:0.15288236795052068,
     -using learning rate from the fourth equality:0.012265035733508785.


From the results it is far obvious that using the learning rate from the fourth inequality resulted in the closest point to x* which should not come at any surprise, since it is adaptive on every step.

# Exercise 6

In [29]:
a = np.array([
    [3, 10, 30],
    [0.1, 10, 35],
    [3, 10, 30],
    [0.1, 10, 35]
    ])
c = np.array([1, 1.2, 3, 3.2])
p = np.array([
    [0.36890, 0.1170, 0.2673],
    [0.46990, 0.4387, 0.7470],
    [0.10910, 0.8732, 0.5547],
    [0.03815, 0.5743, 0.8828]
    ])

In [30]:
def f(z):
    final_sum=0
    for i in range(4):
        exponent_sum=0
        for j in range(3):
            exponent_sum-=a[i][j]*(z[j]-p[i][j])**2
        final_sum-=c[i]*math.exp(exponent_sum)
    return final_sum

def f_gradient(z):
    
    final_sum=[0,0,0]
    for i in range(4):
        
        exponent_sum=0
        for j in range(3):
            exponent_sum-=a[i][j]*(z[j]-p[i][j])**2
        
        k=0
        final_sum[k]+=c[i]*math.exp(exponent_sum)*2*a[i][k]*(z[k]-p[i][k]) 
        k=1
        final_sum[k]+=c[i]*math.exp(exponent_sum)*2*a[i][k]*(z[k]-p[i][k])  
        k=2
        final_sum[k]+=c[i]*math.exp(exponent_sum)*2*a[i][k]*(z[k]-p[i][k])  
        
    return np.array(final_sum)

In [31]:
def pgd(gamma, z, niter):
    #all_points=[z]
    for i in range(1,niter):
        z=z-gamma*f_gradient(z)
        #z=np.array(pd( z_new[0], z_new[1], z_new[2] ))
        #all_points.append(z)
    return z

#for i in all_points:
#    print(f'New points:{list(i)}, the value of f at them:{f(i)}')

In [32]:
actual_min=-3.86278214782076
alpha=0.01; z=np.array([0.5,0.5,0.5]); niter=1000
score=f(pgd(alpha, np.array([0.5,0.5,0.5]), niter))
print(f'Using alpha={alpha} and {niter} iterations we got {score},\n \
    which is {abs(score-actual_min)} away from the real minimum of f')

Using alpha=0.01 and 1000 iterations we got -3.862782147818607,
     which is 2.1529444893531036e-12 away from the real minimum of f


In [33]:
alpha=0.1; z=np.array([0.5,0.5,0.5]); niter=1000
score=f(pgd(alpha, np.array([0.5,0.5,0.5]), niter))
print(f'Using alpha={alpha} and {niter} iterations we got {score},\n \
    which is {abs(score-actual_min)} away from the real minimum of f')

Using alpha=0.1 and 1000 iterations we got -1.8965338597606195e-16,
     which is 3.86278214782076 away from the real minimum of f


In [34]:
alpha=0.01; z=np.array([0.5,0.5,0.5]); niter=100
score=f(pgd(alpha, np.array([0.5,0.5,0.5]), niter))
print(f'Using alpha={alpha} and {niter} iterations we got {score},\n \
    which is {abs(score-actual_min)} away from the real minimum of f')

Using alpha=0.01 and 100 iterations we got -3.855864223369109,
     which is 0.006917924451650848 away from the real minimum of f


Managed to get pretty close to the real x* by using alpha=0.01 and 1000 iterations.